# **Análise 01: Análise de Comportamento de Compra de Consumidores do E-commerce Brasileiro**
----

## **Introdução**

### Objetivo
A finalidade desta análise é examinar os dados dos consumidores do E-commerce brasileiro Olist, com o intuito de identificar os padrões de comportamento de compra e responder a diversas questões relacionadas ao negócio. Para alcançar esse objetivo, serão utilizadas diversas técnicas de análise de dados, incluindo a segmentação de clientes e análise de associação.


### Perguntas de Negócio
Está análise se propõe a responder as seguintes demandas:
* Quais são os produtos mais vendidos?
* Quais são os produtos que são geralmente comprados juntos?
* Qual é o valor médio de cada compra?
* Quais são os clientes mais valiosos?
* Qual é a taxa de recompra dos clientes?
* Quais são as preferências de compra dos clientes?
* Como as vendas variam ao longo do tempo?
* Como as vendas variam de acordo com a localização geográfica?
* Como o preço afeta o comportamento de compra dos clientes?
* Quais são as tendências de compra ao longo do tempo?

## Bibliotecas e Definições

Lista de todas as bibliotecas utilizadas, padronização de recursos e definições de funções.

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Etapa I - Entendimento dos dados e limpeza

Etapa de entendimento dos datasets utilizados, onde serão aplicados as técnicas de ETL (Extract, Transform and Load), bem como EDA (Exploratory Data Analysis)

### Coleta dos Dados
Carregamento dos dados que serão analisados

In [23]:
# Carregando dados dos Clientes
df_clientes = pd.read_csv('dados/olist_customers_dataset.csv', sep=',')
df_clientes.head(3)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP


In [25]:
# Carregando dados dos Pedidos
# utilizado argumento parse_dates para garantir que todas as colunas de datas estejam com o tipo datetime
df_pedidos = pd.read_csv('dados/olist_orders_dataset.csv', sep=',', parse_dates=['order_purchase_timestamp','order_approved_at', \
                                                                                 'order_delivered_carrier_date', 'order_delivered_customer_date', \
                                                                                 'order_estimated_delivery_date'])
df_pedidos.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04


In [29]:
# Carregandos dados dos Itens dos Pedidos
# utilizado argumento parse_dates para garantir que todas as colunas de datas estejam com o tipo datetime
df_itens_pedidos = pd.read_csv('dados/olist_order_items_dataset.csv', sep=',', parse_dates=['shipping_limit_date'])
df_itens_pedidos.head(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87


### Entendendo os datasets
Utilizando técnicas de visualizações e resumos estátisticos para entender os dados e as variáveis de cada dataset

In [31]:
# Informações dos datasets

print(df_clientes.info(), '\n')
print(df_pedidos.info(), '\n')
print(df_itens_pedidos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_pur

In [33]:
# Verificando se há dados duplicados

if df_clientes.duplicated().sum() + df_pedidos.duplicated().sum() + df_itens_pedidos.duplicated().sum() == 0:
    print("Não há dados duplicados")
else:
    print("Existem dados duplicados que serão tratados na etapa de limpeza")

Não há dados duplicados


In [58]:
# Resumo estatístistico - Clientes
# incluindo todas as colunas

df_clientes.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
customer_id,99441,99441,06b8999e2fba1a1fbc88172c00ba8bc7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
customer_unique_id,99441,96096,8d50f5eadf50201ccdcedfb9e2ac8455,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
customer_zip_code_prefix,99441.0,NaN,NaN,NaN,35137.474583,29797.938996,1003.0,11347.0,24416.0,58900.0,99990.0
customer_city,99441,4119,sao paulo,15540,NaN,NaN,NaN,NaN,NaN,NaN,NaN
customer_state,99441,27,SP,41746,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
# Resumo estatístistico - Pedidos
# tratando dodos de data como numéricos

df_pedidos.describe(include='all',datetime_is_numeric=True).T

,count,unique,top,freq,mean,min,25%,50%,75%,max
order_id,99441,99441,e481f51cbdc54678b7cc49136f2d6af7,1,NaT,NaT,NaT,NaT,NaT,NaT
customer_id,99441,99441,9ef432eb6251297304e76186b10a928d,1,NaT,NaT,NaT,NaT,NaT,NaT
order_status,99441,8,delivered,96478,NaT,NaT,NaT,NaT,NaT,NaT
order_purchase_timestamp,99441,NaN,NaN,NaN,2017-12-31 08:43:12.776581120,2016-09-04 21:15:19,2017-09-12 14:46:19.000000000,2018-01-18 23:04:36.000,2018-05-04 15:42:16.000000000,2018-10-17 17:30:18
order_approved_at,99281,NaN,NaN,NaN,2017-12-31 18:35:24.098800128,2016-09-15 12:16:38,2017-09-12 23:24:16.000000000,2018-01-19 11:36:13.000,2018-05-04 20:35:10.000000000,2018-09-03 17:40:06
order_delivered_carrier_date,97658,NaN,NaN,NaN,2018-01-04 21:49:48.138278656,2016-10-08 10:34:01,2017-09-15 22:28:50.249999872,2018-01-24 16:10:58.000,2018-05-08 13:37:45.000000000,2018-09-11 19:48:28
order_delivered_customer_date,96476,NaN,NaN,NaN,2018-01-14 12:09:19.035542272,2016-10-11 13:46:32,2017-09-25 22:07:22.249999872,2018-02-02 19:28:10.500,2018-05-15 22:48:52.249999872,2018-10-17 13:22:46
order_estimated_delivery_date,99441,NaN,NaN,NaN,2018-01-24 03:08:37.730111232,2016-09-30 00:00:00,2017-10-03 00:00:00.000000000,2018-02-15 00:00:00.000,2018-05-25 00:00:00.000000000,2018-11-12 00:00:00


In [60]:
# Resumo estatístistico - Itens dos Pedidos
# tratando dodos de data como numéricos

df_itens_pedidos.describe(include='all', datetime_is_numeric=True).T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
order_id,112650,98666,8272b63d03f5f79c56e9e4120aec44ef,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
order_item_id,112650.0,NaN,NaN,NaN,1.197834,1.0,1.0,1.0,1.0,21.0,0.705124
product_id,112650,32951,aca2eb7d00ea1a7b8ebd4e68314663af,527,NaN,NaN,NaN,NaN,NaN,NaN,NaN
seller_id,112650,3095,6560211a19b47992c3666cc44a7e94c0,2033,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shipping_limit_date,112650,NaN,NaN,NaN,2018-01-07 15:36:52.192685312,2016-09-19 00:15:34,2017-09-20 20:57:27.500000,2018-01-26 13:59:35,2018-05-10 14:34:00.750000128,2020-04-09 22:35:08,NaN
price,112650.0,NaN,NaN,NaN,120.653739,0.85,39.9,74.99,134.9,6735.0,183.633928
freight_value,112650.0,NaN,NaN,NaN,19.99032,0.0,13.08,16.26,21.15,409.68,15.806405


### Limpeza e Transformação dos Dados
Antes de iniciar a análise é preciso preparar os dados e garantir a qualidade deles, para isso são aplicadas técnicas de limpeza e transformação dos dados, onde são identificados e tratados os valores faltantes, outliers, duplicados e valores inconsistentes ou inválidos. Além de realizar junções, agrupamentos, criar campos calculados, entre outras transformações identificadas como necessárias.

### Análise Exploratório dos Dados

## **Etapa II - Análise de Vendas**

## **Etapa III - Análise de Clientes**

## **Etapa IV - Análise geográfica e de preços**

## **Respostas Questionamentos**

## **Conclusão**

## **Referências**